# Read Initial Data

In [71]:
import pandas as pd
time = []
open = []
high = []
low = []
close = []
volume = []
size = -1

def read_df(file_name):

    df = pd.read_feather(file_name)
    df = df.dropna()    # Dropping NA values, so that talib works correctly
    global size
    size = df.shape[0]
    
    global time
    time = df['time'].to_numpy()

    global open
    open = df['open'].to_numpy()

    global high
    high = df['high'].to_numpy()

    global low
    low = df['low'].to_numpy()

    global close
    close = df['close'].to_numpy()

    global volume
    volume = df['volume'].to_numpy()

    return

read_df("data-BNBUSDT1h.feather")

# Create Indicators

In [72]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import talib as ta    # https://github.com/mrjbq7/ta-lib

ma_direction_period = 28
ma_direction = ta.EMA(close, timeperiod=ma_direction_period)
atr_direction_period = 14
atr_direction = ta.ATR(high, low, close, timeperiod=atr_direction_period)

# Generate Trades

In [73]:
in_trade = 0    # +1 = long_entry, -1 = shor_entry
entry_price = -1.0
trades_long = []
trades_short = []

for i in range(size):
    # Long Entry
    if (ma_direction[i] - close[i] > 4.0*atr_direction[i]
        and in_trade == 0.0):
        in_trade = +1.0
        entry_price = close[i]
    # Long Exit
    if (ma_direction[i] - close[i] < 0.0
        and in_trade == 1.0):
        in_trade = 0.0
        trades_long.append(close[i]/entry_price)

    # Short Entry
    if (ma_direction[i] - close[i] < -4.0*atr_direction[i]
        and in_trade == 0.0):
        in_trade = -1.0
        entry_price = close[i]
    # Short Exit
    if (ma_direction[i] - close[i] > 0.0
        and in_trade == -1.0):
        in_trade = 0.0
        trades_short.append(entry_price/close[i])


# Calculate Results

In [74]:
long_res = 1.0
for k in trades_long:
    long_res *= k

print("Longs result: ", long_res)
average_long_trades_res = np.sum(trades_long)/len(trades_long)
print("Average Long Trade: ", average_long_trades_res)
DD_long = 1.0 - np.min(trades_long)
print("Highest DD for Longs: ", DD_long)
print()

short_res = 1.0
for k in trades_short:
    short_res *= k

print("Shorts result: ", short_res)
average_short_trades_res = np.sum(trades_short)/len(trades_short)
print("Average Short Trade: ", average_short_trades_res)
DD_short = 1.0 - np.min(trades_short)
print("Highest DD for Shorts: ", DD_short)
print()


DD_highest = None
if DD_long > DD_short:
    DD_highest = DD_long
else: DD_highest = DD_short

score = ((average_long_trades_res + average_short_trades_res) / 2) / DD_highest
print("Score: ", score)

Longs result:  1.1932475343208822
Average Long Trade:  1.0058177152451184
Highest DD for Longs:  0.2489020352179997

Shorts result:  0.16694511027625009
Average Short Trade:  0.9839752741179365
Highest DD for Shorts:  0.4806757383373651

Score:  2.0697872085718907


# Functions for Optimizations

# Optimize Results

In [75]:
import matplotlib.pyplot as plt

# Strategy Function
def strategy_ma(ma_direction_period, atr_direction_period, atr_direction_coeff, show_money = False):
    # Generate Indicators
    ma_direction = ta.EMA(close, timeperiod=ma_direction_period)
    atr_direction = ta.ATR(high, low, close, timeperiod=atr_direction_period)

    # Calculate Trades
    in_trade = 0    # +1 = long_entry, -1 = shor_entry
    entry_price = -1.0
    trades_long = []
    trades_short = []
    # Parse dataframe
    for i in range(size):
        # Long Entry
        if (ma_direction[i] - close[i] > atr_direction_coeff*atr_direction[i]
            and in_trade == 0.0):
            in_trade = +1.0
            entry_price = close[i]
        # Long Exit
        if (ma_direction[i] - close[i] < 0.0
            and in_trade == 1.0):
            in_trade = 0.0
            trades_long.append(close[i]/entry_price)

        # Short Entry
        if (ma_direction[i] - close[i] < -atr_direction_coeff*atr_direction[i]
            and in_trade == 0.0):
            in_trade = -1.0
            entry_price = close[i]
        # Short Exit
        if (ma_direction[i] - close[i] > 0.0
            and in_trade == -1.0):
            in_trade = 0.0
            trades_short.append(entry_price/close[i])

    # Calculating Stats
    # DD Long
    cumulitive_return_long = 1.0
    current_high = 1.0
    current_dd = 0.0
    current_dd_lowest = 0.0
    dd_long = []
    for trade in trades_long:
        cumulitive_return_long *= trade
        if current_high - cumulitive_return_long > 0.0:             # Portfolio goes lower
            current_dd = (current_high - cumulitive_return_long) / current_high
            if current_dd > current_dd_lowest:                      # Porfolio found new lowest point
                current_dd_lowest = current_dd
        if cumulitive_return_long > current_high and current_dd_lowest != 0.0:     # Portfolio goes up after DD
            dd_long.append(current_dd_lowest)
            current_dd = 0.0
            current_dd_lowest = 0.0
        if cumulitive_return_long > current_high:     # Portfolio goes up
            current_high = cumulitive_return_long
    if current_dd_lowest != 0.0: dd_long.append(current_dd_lowest)

    # DD Short
    cumulitive_return_short = 1.0
    current_high = 1.0
    current_dd = 0.0
    current_dd_lowest = 0.0
    dd_short = []
    for trade in trades_short:
        cumulitive_return_short *= trade
        if current_high - cumulitive_return_short > 0.0:             # Portfolio goes lower
            current_dd = (current_high - cumulitive_return_short) / current_high
            if current_dd > current_dd_lowest:                      # Porfolio found new lowest point
                current_dd_lowest = current_dd
        if cumulitive_return_short > current_high and current_dd_lowest != 0.0:     # Portfolio goes up after DD
            dd_short.append(current_dd_lowest)
            current_dd = 0.0
            current_dd_lowest = 0.0
        if cumulitive_return_short > current_high:     # Portfolio goes up
            current_high = cumulitive_return_short
    if current_dd_lowest != 0.0: dd_short.append(current_dd_lowest)

    # Finding Average Trade Size
    if len(trades_long) + len(trades_short) > 20: # If enough trades happened
        average_long_trades_res = np.sum(trades_long)/len(trades_long)
        average_short_trades_res = np.sum(trades_short)/len(trades_short)
        if show_money:
            print("Long Trades: ", trades_long)
            print("Average Long Trade: ", average_long_trades_res)
            print("Amount of Long Trades: ", len(trades_long))
            print("DD Long: ", dd_long)
            
            print("Short Trades: ", trades_short)
            print("Average Short Trade: ", average_short_trades_res)
            print("Amount of Short Trades: ", len(trades_short))
            print("DD Short: ", dd_short)
        # (Longs/DD_long + Shorts/DD_short) / 2
        score = (average_long_trades_res/np.max(dd_long) + average_short_trades_res/np.max(dd_short)) / 2    # Gives same weight to long/short not overall trades
        return -score
    else: return None


In [76]:
# Generate new candidate solution
# Random Initialization
def get_rand_candidate_solution_ma():
    params = []
    params.append(np.random.randint(8, 192))
    params.append(np.random.randint(8, 96))
    params.append(np.random.uniform(-1.0, 5.0))
    return params

# Small pertrubation from found solution (Is not working, obviously)
def get_candidate_solution_ma(params):
    params[0] += np.random.randint(-4, 5)
    if params[0] < 8: params[0] = 8
    params[1] += np.random.randint(-4, 5)
    if params[1] < 4: params[1] = 4
    params[2] += np.random.uniform(-0.12, 0.12)
    return params

# Simulated Annealing

In [77]:
import copy

# Cooling Schedules
temp_max = 1.0
temp_min = 0.0
outer_loop = 100
inner_loop = 20

def cool_lin_a(step):
    return temp_min + (temp_max - temp_min) * ((outer_loop - step)/outer_loop)
def cool_quad_a(step):
    return temp_min + (temp_max - temp_min) * ((outer_loop - step)/outer_loop)**2
def cool_exp_a(step):
    return temp_min + (temp_max - temp_min) * ((outer_loop - step)/outer_loop)**np.sqrt(step)
def cool_cos_a(step):
    return temp_min + 0.5*(temp_max - temp_min) * (1 + np.cos(step*np.pi/outer_loop))

# Simulated Annealing
def annealing_solution():

    step = 0
    temp = temp_max

    # Generate Init Candidate Solution
    candidate_params = get_rand_candidate_solution_ma()
    print("Initial Guess: ", candidate_params)
    best_params = copy.deepcopy(candidate_params)
    current_energy = strategy_ma(candidate_params[0], candidate_params[1], candidate_params[2], show_money=True)
    print("Initial Score: ", current_energy)
    print()
    best_solution = current_energy

    # Annealing Schedule
    while step < outer_loop:
        
        step += 1
        mc_step = 0

        while mc_step < inner_loop:

            mc_step += 1

            # Generate Candidate Solution
            candidate_params = get_rand_candidate_solution_ma()
            candidate_energy = strategy_ma(candidate_params[0], candidate_params[1], candidate_params[2])
            if candidate_energy:    # If any solution found
                delta_energy = candidate_energy - current_energy    # Positive if solution is better, and is always accepted
                if candidate_energy < best_solution:
                    best_solution = candidate_energy
                    best_params = copy.deepcopy(candidate_params)

                if np.random.random() < np.exp(-delta_energy / temp):   # Value is >1 if solution is better anyway, otherwise Metropolis is used
                    current_energy = candidate_energy
            
        # Cooling Function
        temp = cool_exp_a(step)
        
    return best_params

found_sol = annealing_solution()
print("Found Params: ", found_sol)
print(strategy_ma(found_sol[0], found_sol[1], found_sol[2], show_money=True))

Initial Guess:  [184, 60, 2.9978182932898556]
Long Trades:  [1.0169811320754718, 1.027761499045816, 0.7050590853524747, 0.9879560715152497, 1.0579151381628857, 1.1110021252247835, 1.0810515541032657, 1.0489366374432, 1.0118503421167564, 0.9806776132331375, 1.0581325276340012, 0.9861640667898934, 1.0597922889321238, 0.974962164458487, 1.0147588192944565, 0.974805710109041, 1.007106360961097, 0.8132545731707318, 1.026130470752033, 1.0542830579385538, 0.9312186468804242, 1.0313595159726883, 1.048478898833457, 1.009209845075471, 0.9865715398651004, 0.5818487218518713, 1.0567975121750866, 1.0883900867994187, 0.9739280884575611, 1.0799973561876226, 1.0885601730264174, 1.0000674365674787, 1.0442698722342314, 1.0413633301013954, 1.0377819099757954, 1.0224442111679253, 1.0362787255632986, 1.0419656121520622, 1.0290750121435015, 1.0006303767428062, 1.0564841610755675, 1.0271748319049043, 1.0381250557311432, 1.0500590970991033, 1.0302879668913218, 1.0092149576472143, 1.0052024782033024, 0.9689546

/tmp/ipykernel_20060/2117758087.py:52: RuntimeWarning: overflow encountered in exp
  if np.random.random() < np.exp(-delta_energy / temp):   # Value is >1 if solution is better anyway, otherwise Metropolis is used


Found Params:  [14, 8, 3.562699033379002]
Long Trades:  [1.06944670937682, 1.043556701030928, 1.0516917628747475, 0.9482175583801361, 0.9030350877192983, 1.036677096370463, 1.039265569630718, 1.0284817421774144, 1.0233889027121175, 1.0471404642258915]
Average Long Trade:  1.0190901594498536
Amount of Long Trades:  10
DD Long:  [0.14372627399121488]
Short Trades:  [0.8161711140490516, 1.048457062581276, 1.0356052899287893, 0.9939546969666606, 0.9974205955524352, 1.0450832131964263, 1.0365291973961288, 0.9957314826188429, 0.973330468941812, 0.988012119615334, 1.0007783265978147, 1.0006930837091104, 1.0364615427707133, 0.7673613940308261, 1.0017296341126378, 1.0143234682967177, 0.9924690693921463, 1.0155135520684737, 1.0177184466019418]
Average Short Trade:  0.9882812504435337
Amount of Short Trades:  19
DD Short:  [0.2741277040680937]
-5.3478393673750535


# Pyswarms

In [78]:
# Import modules
import numpy as np

# Import PySwarms
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)

# Perform optimization
cost, pos = optimizer.optimize(fx.sphere, iters=1000)

2022-12-25 02:43:37,815 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=3.99e-44
2022-12-25 02:43:39,650 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 3.9939864794188724e-44, best pos: [ 1.82199969e-22 -8.21159920e-23]
